In [1]:
from   symcircuit.symbolic_solver.domains    import Circuit, ExperimentResult
from   symcircuit.symbolic_solver.solver     import Circuit_Solver, Impedance_Analyzer
from   symcircuit.demo.multiple_feedback     import Customized_Current_Mode_Multiple_Feedback as TIA_CMMF
import sympy

# Manual Runs

In [2]:
circuit = TIA_CMMF.circuit
circuit.hasSolution()

True

In [3]:
circuit.impedances, circuit.impedancesToDisconnect

([Z_1, Z_2, Z_3, Z_4, Z_5, Z_6], [Z_1, Z_2, Z_3, Z_4, Z_5, Z_6])

In [4]:
for i, eq in enumerate(circuit.nodal_equations, 1):
    print(f"{i} - {eq}")

1 - Eq(Iin, -Va/Z_2 - Vo1/Z_1)
2 - Eq(0, Va/Z_5 - Va/Z_4 - Va/Z_2 + (-Va + Vo1)/Z_3)
3 - Eq(Va/Z_5, -Vo2/Z_6)


In [5]:
circuit.solve_for

[Vo1, Vo2, Va, Iin]

## Circuit Solver

In [6]:
solver = Circuit_Solver(circuit=circuit, _output= sympy.symbols("Vo2 0"), _input=sympy.symbols("Iin 0"), transmissionMatrixType="NA")

In [7]:
solver.impedances

[Z_1, Z_2, Z_3, Z_4, Z_5, Z_6]

In [8]:
solver.impedancesToDisconnect

[Z_1, Z_2, Z_3, Z_4, Z_5, Z_6]

In [9]:
solver.solve()

====Solving the Circuit====
--- Computing the possible impedance connections for [Z_1, Z_2, Z_3, Z_4, Z_5, Z_6] ---
--- Impedance connections stored in Circuit_Solver.impedanceConnections---
 ----------------------------
1 - Solving for (Vo2 - 0) / (Iin - 0)
2 - Intermediate Variables: [Vo1, Vo2, Va, Iin]
 ----------------------------
3 - solved the base transfer function (symbolic [T])
-- Single-ended input/output
processing combo index 1
processing combo index 2
processing combo index 3
processing combo index 4
processing combo index 5
processing combo index 6
=====*Circuit Solved*=====


In [10]:
solver.solutions

[{Va: -Iin*Z_1*Z_2*Z_4*Z_5/(Z_1*Z_4*Z_5 - Z_2*Z_3*Z_4 + Z_2*Z_3*Z_5 + Z_2*Z_4*Z_5 + Z_3*Z_4*Z_5),
  Vo1: Iin*(Z_1*Z_2*Z_3*Z_4 - Z_1*Z_2*Z_3*Z_5 - Z_1*Z_2*Z_4*Z_5 - Z_1*Z_3*Z_4*Z_5)/(Z_1*Z_4*Z_5 - Z_2*Z_3*Z_4 + Z_2*Z_3*Z_5 + Z_2*Z_4*Z_5 + Z_3*Z_4*Z_5),
  Vo2: Iin*Z_1*Z_2*Z_4*Z_6/(Z_1*Z_4*Z_5 - Z_2*Z_3*Z_4 + Z_2*Z_3*Z_5 + Z_2*Z_4*Z_5 + Z_3*Z_4*Z_5)}]

In [11]:
symcircuit_sol = solver.baseHs
symcircuit_sol

Z_1*Z_2*Z_4*Z_6/(Z_1*Z_4*Z_5 - Z_2*Z_3*Z_4 + Z_2*Z_3*Z_5 + Z_2*Z_4*Z_5 + Z_3*Z_4*Z_5)

## Impedance Analyzer

In [12]:
analysis = Impedance_Analyzer("VLSI_CMMF", circuit_solver=solver)
print(analysis.isCircuitSolved())
analysis

True


In [13]:
for block in analysis.impedance_blocks:
    print(f"{block.name} - {block.symbol} - allowed connections {block.allowedConnections}")

1 - Z_1 - allowed connections [R_1, 1/(C_1*s), R_1 + 1/(C_1*s), R_1/(C_1*R_1*s + 1)]
2 - Z_2 - allowed connections [R_2, 1/(C_2*s), R_2 + 1/(C_2*s), R_2/(C_2*R_2*s + 1)]
3 - Z_3 - allowed connections [R_3, 1/(C_3*s), R_3 + 1/(C_3*s), R_3/(C_3*R_3*s + 1)]
4 - Z_4 - allowed connections [R_4, 1/(C_4*s), R_4 + 1/(C_4*s), R_4/(C_4*R_4*s + 1)]
5 - Z_5 - allowed connections [R_5, 1/(C_5*s), R_5 + 1/(C_5*s), R_5/(C_5*R_5*s + 1)]
6 - Z_6 - allowed connections [R_6, 1/(C_6*s), R_6 + 1/(C_6*s), R_6/(C_6*R_6*s + 1)]


In [14]:
analysis.computeTFs(comboKey="Z1_Z2_Z3_Z4_Z5_Z6")

combo key = Z1_Z2_Z3_Z4_Z5_Z6


Getting the TFs (CG): 100%|██████████| 4096/4096 [00:48<00:00, 83.81combo/s] 

Number of transfer functions found: 4096


In [15]:
analysis.classifier.classifyBiQuadFilters()

Computing Biquad filter parameters: 100%|██████████| 4096/4096 [00:49<00:00, 83.35filter/s] 


In [16]:
classifications = analysis.classifier.summarizeFilterType()

summarizing for filters in ['AP', 'BP', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 120
BS : 0
GE : 0
HP : 28
LP : 120
X-INVALID-NUMER : 434
X-INVALID-ORDER : 3190
X-INVALID-WZ : 204
X-PolynomialError : 0


In [17]:
analysis.reportSummary("CMMF_BiQuad", "Z1_Z2_Z3_Z4_Z5_Z6")

LaTeX report generated and saved to: Runs/VLSI_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6_summary.tex


In [18]:
analysis.classifier.classifyFilter(filterOrder="FirstOrder")

Computing filter parameters for FirstOrder: 100%|██████████| 4096/4096 [00:28<00:00, 145.33filter/s]


In [19]:
classifications = analysis.classifier.summarizeFilterType()

summarizing for filters in ['AP', 'BP', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 73
BS : 0
GE : 0
HP : 28
LP : 52
X-INVALID-NUMER : 0
X-INVALID-ORDER : 3943
X-INVALID-WZ : 0
X-PolynomialError : 0


In [20]:
analysis.reportSummary("CMMF_FirstOrder_no_stability_check", "Z1_Z2_Z3_Z4_Z5")

LaTeX report generated and saved to: Runs/VLSI_CMMF/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5_summary.tex


In [21]:
analysis.classifier.validate_stability(fType="BP")

100%|██████████| 4096/4096 [01:53<00:00, 36.07filter/s] 

summarizing for filters in ['AP', 'BP', 'BP-UNSTABLE-ZERO', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 72
BP-UNSTABLE-ZERO : 1
BS : 0
GE : 0
HP : 28
LP : 52
X-INVALID-NUMER : 0
X-INVALID-ORDER : 3943
X-INVALID-WZ : 0
X-PolynomialError : 0


[FilterClassification(valid=False, fType=BP-UNSTABLE-ZERO, parameters={'wo': -(C_6*R_1*R_4 + C_6*R_2*R_3 + C_6*R_2*R_4 + C_6*R_3*R_4)/(C_5*C_6*R_2*R_3*R_4), 'wz': 1/(C_6*R_6), 'K': -R_1*R_6/R_3})zCombo=(R_1, R_2, R_3, R_4, 1/(C_5*s), R_6 + 1/(C_6*s)), transferFunc=(C_5*C_6*R_1*R_2*R_4*R_6*s + C_5*R_1*R_2*R_4)/(-C_5*C_6*R_2*R_3*R_4*s + C_6*R_1*R_4 + C_6*R_2*R_3 + C_6*R_2*R_4 + C_6*R_3*R_4), ]

In [22]:
analysis.reportSummary("CMMF_FirstOrder_BP_stability_check", "Z1_Z2_Z3_Z4_Z5")


LaTeX report generated and saved to: Runs/VLSI_CMMF/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5_summary.tex


In [23]:
analysis.reportType(experimentName="CMMF_FirstOrder_BP_stability_check_BP_ONLY", fType="BP", Z_arr="Z1_Z2_Z3_Z4_Z5")


LaTeX report saved to: Runs/VLSI_CMMF/CMMF_FirstOrder_BP_stability_check_BP_ONLY_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_BP_stability_check_BP_ONLY_Z1_Z2_Z3_Z4_Z5_BP.tex


In [24]:
analysis.compilePDF()


=== Compiling the reports to PDF ===
----** compiling Runs/VLSI_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6_summary.tex **----
PDF generated: Runs/VLSI_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_Z6_summary.pdf
----** compiling Runs/VLSI_CMMF/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/VLSI_CMMF/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_no_stability_check_Z1_Z2_Z3_Z4_Z5_summary.pdf
----** compiling Runs/VLSI_CMMF/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/VLSI_CMMF/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_BP_stability_check_Z1_Z2_Z3_Z4_Z5_summary.pdf
----** compiling Runs/VLSI_CMMF/CMMF_FirstOrder_BP_stability_check_BP_ONLY_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_BP_stability_check_BP_ONLY_Z1_Z2_Z3_Z4_Z5_BP.tex **----
P

# Automated Search

In [25]:
from   symcircuit.symbolic_solver.solver     import run_experiment

circuit = TIA_CMMF.circuit


experiment_result = run_experiment("VLSI_CMMF_Automated",
               T_type="NA",
               circuit=circuit,
               minNumOfActiveImpedances=3,
               maxNumOfActiveImpedances=5,
               impedanceKeysOverwrite= [
                   "Z1_Z2_Z3_Z4_Z5_Z6",
                   "Z2_Z3_Z4_Z5_Z6",
                   "Z1_Z3_Z4_Z5_Z6",
                   "Z1_Z2_Z3_Z5_Z6",
                   "Z3_Z4_Z5_Z6",
                   "Z2_Z3_Z5_Z6",
                   "Z1_Z3_Z5_Z6",
                   "Z3_Z5_Z6",
                   ],
               outputFrom= ["Vo2", "0"],
               inputFrom=["Iin", "0"],
               loadHistory=True
               )



!!! could not ressolve the path to the pkl file (found 0)

Solving the circuit for the first time
====Solving the Circuit====
--- Computing the possible impedance connections for [Z_1, Z_2, Z_3, Z_4, Z_5, Z_6] ---
--- Impedance connections stored in Circuit_Solver.impedanceConnections---
 ----------------------------
1 - Solving for (Vo2 - 0) / (Iin - 0)
2 - Intermediate Variables: [Vo1, Vo2, Va, Iin]
 ----------------------------
3 - solved the base transfer function (symbolic [T])
-- Single-ended input/output
processing combo index 1
processing combo index 2
processing combo index 3
processing combo index 4
processing combo index 5
processing combo index 6
=====*Circuit Solved*=====

=== ExperimentResult saved successfully to Runs/VLSI_CMMF_Automated_NA/results_circuit_solution.pkl - 6.641 kb ===

** Loading the circuit history
could not ressolve the path to the pkl file (found 0)
** Found 0 keys already computed
Experiment keys: ['Z1_Z2_Z3_Z4_Z5_Z6', 'Z2_Z3_Z4_Z5_Z6', 'Z1_Z3_Z4_Z5_

Getting the TFs (CG): 100%|██████████| 4096/4096 [00:46<00:00, 87.50combo/s] 


Number of transfer functions found: 4096


Computing Biquad filter parameters: 100%|██████████| 4096/4096 [00:46<00:00, 88.64filter/s] 


summarizing for filters in ['AP', 'BP', 'BP-UNSTABLE-ZERO', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 120
BP-UNSTABLE-ZERO : 0
BS : 0
GE : 0
HP : 28
LP : 120
X-INVALID-NUMER : 434
X-INVALID-ORDER : 3190
X-INVALID-WZ : 204
X-PolynomialError : 0
LaTeX report generated and saved to: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6_summary.tex **----
PDF generated: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_Z6_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/VLSI_CMMF_Automated_NA/results.pkl - 2375.339 kb ===

--> Running the VLSI_CMMF_Automated_NA Experiment for Z2_Z3_Z4_Z5_Z6 (2/8)

comb

...

summarizing for filters in ['AP', 'BP', 'BP-UNSTABLE-ZERO', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 15
BP-UNSTABLE-ZERO : 0
BS : 0
GE : 0
HP : 0
LP : 15
X-INVALID-NUMER : 32
X-INVALID-ORDER : 184
X-INVALID-WZ : 10
X-PolynomialError : 0
LaTeX report generated and saved to: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6_summary.tex **----
PDF generated: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z1_Z3_Z5_Z6_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/VLSI_CMMF_Automated_NA/results.pkl - 3435.062 kb ===

--> Running the VLSI_CMMF_Automated_NA Experiment for Z3_Z5_Z6 (8/8)

combo key = Z3_Z5_Z6


Getting the TFs (CG): 100%|██████████| 64/64 [00:02<00:00, 26.89combo/s]


Number of transfer functions found: 64


Computing Biquad filter parameters: 100%|██████████| 64/64 [00:00<00:00, 679.44filter/s]


summarizing for filters in ['AP', 'BP', 'BP-UNSTABLE-ZERO', 'BS', 'GE', 'HP', 'LP', 'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ', 'X-PolynomialError']
AP : 0
BP : 0
BP-UNSTABLE-ZERO : 0
BS : 0
GE : 0
HP : 0
LP : 0
X-INVALID-NUMER : 0
X-INVALID-ORDER : 8
X-INVALID-WZ : 0
X-PolynomialError : 56
LaTeX report generated and saved to: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z3_Z5_Z6_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z3_Z5_Z6_summary.tex **----
PDF generated: Runs/VLSI_CMMF_Automated_NA/VLSI_CMMF_Automated_NA_Z3_Z5_Z6/VLSI_CMMF_Automated_NA_Z3_Z5_Z6_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/VLSI_CMMF_Automated_NA/results.pkl - 3445.789 kb ===

<----> END OF EXPERIMENT <---->
Impedance Keys analyzed (count: 8): 
['Z1_Z2_Z3_Z4_Z5_Z6',
 'Z2_Z3_Z4_Z5_Z6',
 'Z1_Z3_Z4_Z5_Z6',
 'Z1_Z2_Z3_Z5_Z6

In [27]:
classifications, tf = experiment_result.get_impedance_key("Z1_Z2_Z3_Z4_Z5_Z6")
tf

Z_1*Z_2*Z_4*Z_6/(Z_1*Z_4*Z_5 - Z_2*Z_3*Z_4 + Z_2*Z_3*Z_5 + Z_2*Z_4*Z_5 + Z_3*Z_4*Z_5)

In [ ]:
type(classifications), type(classifications[-1])

In [ ]:
# df_classifications = experiment_result.flatten_classifications()
# df_classifications

In [ ]:
# df_tfs = experiment_result.flatten_tfs()
# df_tfs

In [28]:
experiment_result.to_csv()

Writting VLSI_CMMF_Automated_NA to csv files
flattened all the classifications to Runs/VLSI_CMMF_Automated_NA/classifications_.csv
flattened all the classifications to Runs/VLSI_CMMF_Automated_NA/tfs_.csv
